In [107]:
import pandas as pd
import numpy as np
import pickle

# NLP feature extraction
import nltk
import regex as re
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.options.display.max_columns = 5_000 # So I can see all the columns rather than just ...

In [142]:
gen = (pd.read_csv(f'../data_gathering/scrape_output/review_batching/dmv_beer_review_text_batch{str(batch)}.csv') for batch in range(48))
res = pd.concat(gen, ignore_index=True)

In [143]:
res.shape

(99156, 5)

In [74]:
res.head()

,beer,user,date,rating,review_text
0,3 Stars #ultrafresh,345552,2019-04-11,4.2,Can from Whole Foods Market Fair Lakes in Fair...
1,3 Stars #ultrafresh,44939,2019-01-25,3.7,Canned. Clouded gold pour. Plenty of citrus. ...
2,3 Stars #ultrafresh,87939,2018-09-20,2.9,Draft at Beer Culture. Murky golden with a whi...
3,3 Stars #ultrafresh,91588,2018-08-18,3.6,Can at home. Lighter pour for a dipa decent si...
4,3 Stars #ultrafresh,142215,2018-07-03,3.7,Clear gold pour with a bright white head and n...


In [144]:
full_data = pd.read_csv('../data/dmv_beer_database_extra_categories.csv')
reviews_plus = pd.merge(left=res, right=full_data, left_on=res['beer'], right_on=full_data['name'])
reviews_plus.head()

,key_0,beer,user,date,rating_x,review_text,abv,address,brewery,country,date_added,google_map_link,link,name,num_ratings,postal_code,rating_y,state,style,town,abv_range,category,year_added,ale
0,3 Stars #ultrafresh,3 Stars #ultrafresh,345552,2019-04-11,4.2,Can from Whole Foods Market Fair Lakes in Fair...,0.083,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,6/3/2016,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-ultrafresh/423469/,3 Stars #ultrafresh,14,20011,3.47,DC,IIPA - Imperial / Double IPA,Washington,8% and more,IPA,2016,1
1,3 Stars #ultrafresh,3 Stars #ultrafresh,44939,2019-01-25,3.7,Canned. Clouded gold pour. Plenty of citrus. ...,0.083,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,6/3/2016,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-ultrafresh/423469/,3 Stars #ultrafresh,14,20011,3.47,DC,IIPA - Imperial / Double IPA,Washington,8% and more,IPA,2016,1
2,3 Stars #ultrafresh,3 Stars #ultrafresh,87939,2018-09-20,2.9,Draft at Beer Culture. Murky golden with a whi...,0.083,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,6/3/2016,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-ultrafresh/423469/,3 Stars #ultrafresh,14,20011,3.47,DC,IIPA - Imperial / Double IPA,Washington,8% and more,IPA,2016,1
3,3 Stars #ultrafresh,3 Stars #ultrafresh,91588,2018-08-18,3.6,Can at home. Lighter pour for a dipa decent si...,0.083,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,6/3/2016,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-ultrafresh/423469/,3 Stars #ultrafresh,14,20011,3.47,DC,IIPA - Imperial / Double IPA,Washington,8% and more,IPA,2016,1
4,3 Stars #ultrafresh,3 Stars #ultrafresh,142215,2018-07-03,3.7,Clear gold pour with a bright white head and n...,0.083,"6400 Chillum Pl, NW",3 Stars Brewing Company,USA,6/3/2016,maps.google.com/maps?f=q&hl=en&geocode=&q=3+St...,www.ratebeer.com/beer/3-stars-ultrafresh/423469/,3 Stars #ultrafresh,14,20011,3.47,DC,IIPA - Imperial / Double IPA,Washington,8% and more,IPA,2016,1


In [145]:
reviews_so_far = reviews_plus.drop(columns=['key_0','name'])

In [146]:
# reviews_so_far = pd.read_csv('../data_gathering/scrape_output/ratebeer_review_text.csv')

# Load in pickled list
with open('../data/beerlist.pickle', 'rb') as f:
    beerlist = pickle.load(f)

In [147]:
reviews_so_far['beer'][0]

'3 Stars #ultrafresh'

In [148]:
reviewlist = pd.DataFrame()
count = 0

for beer in beerlist:
    if count % 300 == 0:
        print(count)
    if reviews_so_far[reviews_so_far['beer'] == beer].shape[0] == 0:
        count += 1
        continue
    else:
        reviewlist = reviewlist.append(reviews_so_far[reviews_so_far['beer'] == beer], ignore_index=True)
        count += 1

0
300
600
900
1200
1500
1800
2100
2400
2700
3000
3300
3600
3900
4200
4500
4800
5100
5400
5700
6000
6300
6600
6900
7200
7500
7800
8100
8400
8700
9000
9300
9600
9900
10200
10500
10800
11100
11400
11700
12000
12300
12600


In [149]:
reviewlist.rename({'rating_x':'rating','rating_y':'agg_rating'}, axis='columns',inplace=True)

In [150]:
# Random review generator
random_review = reviewlist.loc[np.random.choice(range(0,reviewlist.shape[0])),'review_text']
print(random_review)
reviewlist[reviewlist['review_text']==random_review]

On tap at B Side pours a basically opawue, neon orange with fading white head. Nose has mango meat, lemon juice, passion fruit, sweat. Flavor is heavy with the gose, lemon and sweat and yogurt, the mango is there but it's more artificial, some orange soda too. Finishes dry with low acid and a nice, tooth licking minerality.


,beer,user,date,rating,review_text,abv,address,brewery,country,date_added,google_map_link,link,num_ratings,postal_code,agg_rating,state,style,town,abv_range,category,year_added,ale
100187,The Veil Never Said That,38859,2018-08-10,3.5,"On tap at B Side pours a basically opawue, neo...",0.049,1301 Roseneath Road,The Veil Brewing Company,USA,7/28/2018,maps.google.com/maps?f=q&hl=en&geocode=&q=The+...,www.ratebeer.com/beer/the-veil-never-said-that...,7,23230,3.51,VA,Gose,Richmond,4-5%,Sour,2018,0


In [151]:
reviewlist.shape

(104331, 22)

In [94]:
# Instatiate tokenizer, lemmatizer, define function to do both
ws_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def lemmatize_text(text):
    lemma_list = [lemmatizer.lemmatize(w) for w in ws_tokenizer.tokenize(text)]
    return " ".join(lemma_list)

def stem_text(text):
    stem_list = [stemmer.stem(w) for w in ws_tokenizer.tokenize(text)]
    return " ".join(stem_list)

In [95]:
def clean_lemma_df_col(df, col):
    '''
    First removes non-numeric characters from df column then
    performs lemmatization to certain column in dataframe
    '''
    df[col] = df[col].map(lambda x: re.sub("[^a-zA-Z]+"," ", x))
    df['text_lemmatized'] = df[col].apply(lemmatize_text)
    return df

In [96]:
def clean_stem_df_col(df, col):
    '''
    First removes non-numeric characters from df column then
    performs stemming to certain column in dataframe
    '''
    df[col] = df[col].map(lambda x: re.sub("[^a-zA-Z]+"," ", x).lower())
    df['text_stemmed'] = df[col].apply(stem_text)
    return df

In [152]:
beer_review_stem = clean_stem_df_col(reviewlist,'review_text')

In [153]:
beer_review_stem['review_text'][1000]

'pours hazy amber with a white sustained head smells of bananas coriander grainy syruppy tastes sweet and grassy with some cheesy undertones on the finish '

In [154]:
# https://stackoverflow.com/questions/38981885/python-pandas-pivot-table-aggfunc-concatenate-instead-of-np-size-or-np-sum
beer_review_pivot = beer_review_stem.groupby(['beer'])['review_text'].agg({'review_text': 'nunique', 'review_text': ','.join})

/Users/willkarnasiewicz/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [155]:
with open('../data/beer_review_pivot_large.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(beer_review_pivot, f, pickle.HIGHEST_PROTOCOL)

In [156]:
beer_review_pivot.shape

(12061, 1)

In [157]:
beer_review_pivot

,review_text
beer,
1781 Belgiquiose D' or,tap at brewery pours golden slightly hazy arom...
1781 Belsnickel,draft mild pecan thin head cloudy brown thin h...
1781 Conifére,rosehips apricot nose clear copper thin head c...
1781 El Jefe,draft lemon and orange aroma not strong some a...
1781 Kazuno Maharoba,tap at brewery pours slightly hazy golden with...
1781 L'automne,dried leaves subtle mulberry aroma clear garne...
1781 Lafayette Saison (Version 1 - Bottled),bottle opened at brewery for me to try the bre...
1781 Lafayette Saison (Version 2 - Draft),the brewer explained at this version version i...
1781 Persephone,mild cherries aroma hazy tea headless sweet ch...
